In [3]:
using DataFrames, CSV
using JuMP, Gurobi
using LinearAlgebra
using JSON

In [4]:
config = JSON.parsefile("config.json")

#existing_distance = Matrix(CSV.read("inputs/candidate_existing_distance.csv", DataFrame, header = true))
existing_distance_raw = Matrix(CSV.read("inputs/standardized_existing_distance.csv", DataFrame, header = true))
existing_distance = existing_distance_raw[:,2:14]
#attraction_distance = Matrix(CSV.read("inputs/candidate_attraction_distance.csv", DataFrame, header = true))
attraction_distance_raw = Matrix(CSV.read("inputs/standardized_attractions_distance.csv", DataFrame, header = true))
attraction_distance = attraction_distance_raw[:,2:23]
demand_raw = Matrix(CSV.read("inputs/standardized_salem_demand.csv", DataFrame, header = true))
demand = demand_raw[:, 17]
attraction_values_raw = Matrix(CSV.read("inputs/standardized_attractions_weights.csv", DataFrame, header = true))
attraction_values = attraction_values_raw[:,6]
candidate_addresses_raw = Matrix(CSV.read("inputs/candidate_address.csv", DataFrame, header = true));


In [221]:
#raw = Matrix(CSV.read("inputs/candidate_existing_distance.csv", DataFrame, header = false))
#raw_sorted = raw[:,sortperm(raw[1,:])] 
#raw_strings = raw_sorted[2:110,:]
#existing_distance = parse.(Int64,string.(raw_strings))
attraction_distance

109×22 Matrix{Float64}:
 0.694083  0.74502    0.941823   0.647519  …  0.639801  0.355531  0.625909
 0.649063  0.699743   0.896545   0.612275     0.604557  0.491363  0.580632
 0.553877  0.604557   0.801617   0.517089     0.509371  0.396178  0.485446
 0.604042  0.654722   0.851525   0.567254     0.559537  0.446343  0.535612
 0.499081  0.549761   0.746821   0.46255      0.454833  0.482359  0.430908
 0.486733  0.537413   0.734472   0.483903  …  0.476185  0.503712  0.418302
 0.438368  0.489048   0.685851   0.463579     0.434509  0.483388  0.369937
 0.491878  0.542558   0.739618   0.517346     0.488019  0.537155  0.423447
 0.525321  0.576001   0.772804   0.550533     0.521463  0.570342  0.456891
 0.644175  0.694855   0.891657   0.597354     0.589636  0.305366  0.575744
 0.572142  0.622822   0.819882   0.535354  …  0.527637  0.414443  0.503712
 0.501396  0.552076   0.749136   0.464608     0.456891  0.343697  0.432966
 0.518118  0.568798   0.765858   0.48133      0.473612  0.360419  0.449687
 

In [213]:
#raw = Matrix(CSV.read("inputs/standardized_existing_distance.csv", DataFrame, header = true))

109×14 Matrix{Float64}:
   0.0  0.816019  1.0        0.703437  …  0.580837  0.423961  0.394695
   1.0  0.769615  1.0        0.612212     0.587955  0.648333  0.630141
   2.0  0.672062  0.937301   0.514659     0.490666  0.55078   0.532587
   3.0  0.723475  0.988451   0.566072     0.541815  0.602193  0.584001
   4.0  0.616167  0.881142   0.458763     0.432398  0.67839   0.724266
   5.0  0.603247  0.868487   0.398122  …  0.454545  0.700537  0.746413
   6.0  0.55368   0.818655   0.348291     0.433716  0.679708  0.725321
   7.0  0.608521  0.87376    0.403395     0.488557  0.734549  0.780425
   8.0  0.642796  0.907771   0.437671     0.522832  0.768824  0.814701
   9.0  0.764606  1.0        0.652024     0.529424  0.372548  0.343282
  10.0  0.690782  0.956021   0.533378  …  0.509386  0.401286  0.37202
  11.0  0.618276  0.883515   0.460873     0.43688   0.496994  0.478801
  12.0  0.635414  0.900653   0.47801      0.454018  0.514131  0.495939
   ⋮                                   ⋱              

In [222]:
demand_raw

13×18 Matrix{Any}:
  0  516  "T32001"  …  "Existing Stations"
  1  517  "T32002"     "Existing Stations"
  2  518  "T32003"     "Existing Stations"
  3  519  "T32004"     "Existing Stations"
  4  520  "T32005"     "Existing Stations"
  5  521  "T32006"  …  "Existing Stations"
  6  562  "T32008"     "Existing Stations"
  7  570  "T32009"     "Existing Stations"
  8  571  "T32010"     "Existing Stations"
  9  572  "T32011"     "Existing Stations"
 10  573  "T32012"  …  "Existing Stations"
 11  574  "T32013"     "Existing Stations"
 12  575  "T32014"     "Existing Stations"

In [5]:
row_num =  zeros(size(candidate_addresses_raw,1), 1)
for i=1:size(candidate_addresses_raw,1)
    row_num[i] = i
end
row_num
candidate_addresses = [row_num candidate_addresses_raw]

109×5 Matrix{Any}:
   1.0  …  42.5074  -70.9075
   2.0     42.5077  -70.9046
   3.0     42.5076  -70.9025
   4.0     42.5076  -70.8996
   5.0     42.5075  -70.897
   6.0  …  42.5078  -70.8943
   7.0     42.5078  -70.8917
   8.0     42.5079  -70.8891
   9.0     42.508   -70.8867
  10.0     42.5096  -70.9077
  11.0  …  42.5088  -70.9036
  12.0     42.5094  -70.9016
  13.0     42.5099  -70.8996
   ⋮    ⋱           
  98.0     42.5274  -70.8897
  99.0     42.5279  -70.881
 100.0     42.5278  -70.879
 101.0  …  42.529   -70.9081
 102.0     42.5291  -70.9056
 103.0     42.5293  -70.9028
 104.0     42.5291  -70.9001
 105.0     42.5295  -70.8921
 106.0  …  42.5295  -70.8895
 107.0     42.5296  -70.8869
 108.0     42.5294  -70.8813
 109.0     42.5297  -70.8791

In [353]:
demand

13-element Vector{Any}:
 0.3343945738311936
 0.44126379901027796
 0.5131674568294288
 0.8137176869571238
 0.7606512254399579
 0.33096861265875355
 1.0
 0.22663252240717027
 0.14908121950375472
 0.06405509222410631
 0.042876423158113304
 0.0
 0.012042772606152887

In [225]:
existing_distance

109×13 Matrix{Float64}:
 0.816019  1.0        0.703437  0.526787  …  0.580837  0.423961  0.394695
 0.769615  1.0        0.612212  0.426334     0.587955  0.648333  0.630141
 0.672062  0.937301   0.514659  0.32878      0.490666  0.55078   0.532587
 0.723475  0.988451   0.566072  0.380194     0.541815  0.602193  0.584001
 0.616167  0.881142   0.458763  0.282904     0.432398  0.67839   0.724266
 0.603247  0.868487   0.398122  0.304788  …  0.454545  0.700537  0.746413
 0.55368   0.818655   0.348291  0.409723     0.433716  0.679708  0.725321
 0.608521  0.87376    0.403395  0.438725     0.488557  0.734549  0.780425
 0.642796  0.907771   0.437671  0.473001     0.522832  0.768824  0.814701
 0.764606  1.0        0.652024  0.475374     0.529424  0.372548  0.343282
 0.690782  0.956021   0.533378  0.3475    …  0.509386  0.401286  0.37202
 0.618276  0.883515   0.460873  0.274994     0.43688   0.496994  0.478801
 0.635414  0.900653   0.47801   0.292132     0.454018  0.514131  0.495939
 ⋮             

In [226]:
attraction_distance

109×22 Matrix{Float64}:
 0.694083  0.74502    0.941823   0.647519  …  0.639801  0.355531  0.625909
 0.649063  0.699743   0.896545   0.612275     0.604557  0.491363  0.580632
 0.553877  0.604557   0.801617   0.517089     0.509371  0.396178  0.485446
 0.604042  0.654722   0.851525   0.567254     0.559537  0.446343  0.535612
 0.499081  0.549761   0.746821   0.46255      0.454833  0.482359  0.430908
 0.486733  0.537413   0.734472   0.483903  …  0.476185  0.503712  0.418302
 0.438368  0.489048   0.685851   0.463579     0.434509  0.483388  0.369937
 0.491878  0.542558   0.739618   0.517346     0.488019  0.537155  0.423447
 0.525321  0.576001   0.772804   0.550533     0.521463  0.570342  0.456891
 0.644175  0.694855   0.891657   0.597354     0.589636  0.305366  0.575744
 0.572142  0.622822   0.819882   0.535354  …  0.527637  0.414443  0.503712
 0.501396  0.552076   0.749136   0.464608     0.456891  0.343697  0.432966
 0.518118  0.568798   0.765858   0.48133      0.473612  0.360419  0.449687
 

In [227]:
attraction_values

22-element Vector{Any}:
 0.8535697367384464
 1.0
 0.402167146978068
 0.5283888069049109
 0.8463603805568455
 0.7570449986188506
 0.8621694197704257
 0.8433954146326935
 0.755968231778163
 0.45859285575399256
 0.0
 0.6226808891280088
 0.45382457456576775
 0.5629452465022411
 0.1525324732821192
 0.3898999062087589
 0.2750003728506587
 0.4998646003644476
 0.7915084035356388
 0.7254775242798713
 0.29650829186240163
 0.402167146978068

In [6]:
# parameters

N_CANDIDATE_STATIONS = size(existing_distance)[1]
N_NEW_STATIONS = config["N_NEW_STATIONS"]
#DISTANCE_THRESHOLD = config["DISTANCE_THRESHOLD"]
N_CURRENT_STATIONS = size(demand)[1]
N_ATTRACTIONS = size(attraction_distance)[2]

DISTANCE_THRESHOLD = 0.2006853446636252 #need to normalize the 800 meteres value to fit our data

0.2006853446636252

In [4]:
###FIRST VERSION OF MODEL

model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "OutputFlag", 0) 

M = 1_000_000

# @variable(model, x[i=1:50, j=1:15] >= 0) # amount of waste transported
@variable(model, z[i=1:N_CANDIDATE_STATIONS], Bin) # z: binary variable to indicate whether or not station is selected

# @constraint(model, [i=1:50], sum(x[i, j] for j in 1:15) == q[i])
# @constraint(model, [i=1:50, j=1:15], x[i, j] <= M * z[j])

# add constraint that two candidate stations cannot be too close to each other or existing stations

@constraint(model, sum(z[i] for i in 1:N_CANDIDATE_STATIONS) == N_NEW_STATIONS)
@constraint(model, [i=1:N_CANDIDATE_STATIONS, j=1:N_CURRENT_STATIONS], distance[i, j] >=  1000 * z[i])

@objective(model, Max, sum(-distance[i, j] * demand[j] * z[i] for i=1:N_CANDIDATE_STATIONS, j=1:N_CURRENT_STATIONS))

optimize!(model)

optimal_cost = objective_value(model)
stations_chosen = value.(z)


println("Optimal stations: $stations_chosen")


# printing optimal cost
pretty_optimal_cost = round(optimal_cost)
println("Total Cost: $pretty_optimal_cost")

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-16
Optimal stations: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
Total Cost: -3.04235e6


In [41]:
candidate_station_addresses[findall(x -> x==1, stations_chosen)]

5-element Vector{Any}:
 "Unnamed Road, Salem, MA 01970, USA"
 "91 Leach St, Salem, MA 01970, USA"
 "Building 4, Suite 160 Shetland Park, 45 Congress St, Salem, MA 01970, USA"
 "Unnamed Road, Salem, MA 01970, USA"
 "98 Collins St, Salem, MA 01970, USA"

In [29]:
findall(stations_chosen)

LoadError: TypeError: non-boolean (Float64) used in boolean context

In [363]:
###SECOND VERSION OF MODEL

model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "OutputFlag", 0) 

M = 1_000_000

# i - candidate stations
# j - current stations
# k - attractions

@variable(model, z[i=1:N_CANDIDATE_STATIONS], Bin) # z: binary variable to indicate whether or not station is selected

# add constraint that two candidate stations cannot be too close to each other or existing stations

@constraint(model, sum(z[i] for i in 1:N_CANDIDATE_STATIONS) == N_NEW_STATIONS) #can add up to N new stations
@constraint(model, [i=1:N_CANDIDATE_STATIONS, j=1:N_CURRENT_STATIONS], existing_distance[i, j] >= 0.1 * z[i]) #distance between existing and candidate stations must be at least 400 meters CHECK VALUE

@objective(model, Max, 
    sum(-existing_distance[i, j] * z[i] * demand[j] for i=1:N_CANDIDATE_STATIONS, j=1:N_CURRENT_STATIONS) +
    sum(-attraction_distance[i, k] * z[i] * attraction_values[k] * 0 for i=1:N_CANDIDATE_STATIONS, k=1:N_ATTRACTIONS))

optimize!(model)

optimal_cost = objective_value(model)
stations_chosen = value.(z)

println("Optimal stations: $stations_chosen")


# printing optimal cost
pretty_optimal_cost = round(optimal_cost)
println("Total Cost: $pretty_optimal_cost")

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Optimal stations: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Total Cost: -6.0


In [364]:
candidate_addresses[findall(x -> x==1, stations_chosen)]

5-element Vector{Any}:
 39.0
 47.0
 48.0
 54.0
 87.0

In [375]:
###THIRD VERSION OF MODEL

model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "OutputFlag", 0) 

M = 1_000_000

# i - candidate stations
# j - current stations
# k - attractions

#logistic curve
L = 1
w = -100
q = 0.3
alpha = 0#0.0000004*6
beta = 0#6
gamma = 0#0.0000012
kappa = 1#1

@variable(model, z[i=1:N_CANDIDATE_STATIONS], Bin) # z: binary variable to indicate whether or not station is selected

# add constraint that two candidate stations cannot be too close to each other or existing stations

@constraint(model, sum(z[i] for i in 1:N_CANDIDATE_STATIONS) == N_NEW_STATIONS) #can add up to N new stations
@constraint(model, [i=1:N_CANDIDATE_STATIONS, j=1:N_CURRENT_STATIONS], existing_distance[i, j] >= 0.1 * z[i]) #distance between existing and candidate stations must be at least 400 meters CHECK VALUE

@objective(model, Max, 
    alpha * sum(-existing_distance[i, j] * z[i] for i=1:N_CANDIDATE_STATIONS, j=1:N_CURRENT_STATIONS) +
    beta * sum(demand[j] * z[i] * (L/(1+exp(-w*(existing_distance[i,j]-q)))) for i=1:N_CANDIDATE_STATIONS, j=1:N_CURRENT_STATIONS) + #enforce that demand for stations further away from candidate stations count less
    gamma * sum(-attraction_distance[i, k] * z[i] for i=1:N_CANDIDATE_STATIONS, k=1:N_ATTRACTIONS) + 
    kappa * sum(attraction_values[k] * z[i] * (L/(1+exp(-w*(attraction_distance[i,k]-q)))) for i=1:N_CANDIDATE_STATIONS, k=1:N_ATTRACTIONS))

optimize!(model)

optimal_cost = objective_value(model)
stations_chosen = value.(z)

println("Optimal stations: $stations_chosen")


# printing optimal cost
pretty_optimal_cost = round(optimal_cost)
println("Total Cost: $pretty_optimal_cost")

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Optimal stations: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
Total Cost: 48.0


In [377]:
candidate_addresses[findall(x -> x==1, stations_chosen)]

5-element Vector{Any}:
  48.0
  87.0
  88.0
  98.0
 105.0

**OPTIMIZATION FORMULATION:**

Sets:
- candidate stations $i\in \mathcal{I}=\{1,\ldots,n\}$
- existing stations $j\in \mathcal{J}=\{1,\ldots,m\}$
- attractions $k\in \mathcal{K}=\{1,\ldots,q\}$

Decision variable: $z_{i} $ = 1 if candidate station i is built, 0 otherwise

Parameters:
- $e_{ij} $ = distance between candidate station i and existing station j
- $a_{ik} $ = distance between candidate station i and attraction k
- $d_{j} $ = demand at existing station j
- $v_{k} $ = number of google reviews for attraction k
- all parameters are normalized to be between 0 and 1 
    - add epsilon in the denominator to account for distance = 0

Hyperparameters:
- Weights for each term:
    - alpha = weight on distance between candidate station i and existing station j
    - beta = weight on demand at existing station j
    - gamma = weight on distance between candidate station i and attraction k
    - kappa = weight on number of google reviews for attraction k
- Logistic Function:
    - We apply a logistic function to both the demand and google reviews terms
    - This gives more weight to demand/reviews for stations/attractions that are closer to the candidates
    - and less weight to the demand/reviews for stations/attractions that are further from the candidates
    - $\frac{L}{1+e^{-w(distance-q)}} $
    - L = the curve's maximum value
    - w = steepness of the curve
    - q = the distance value of the sigmoid midpoint

$$ 
\max \quad 
\alpha(\sum_{i=1}^{n} \sum_{j=1}^{m} \frac{1}{e_{ij}+\epsilon} * z_i) +
\beta(\sum_{i=1}^{n} \sum_{j=1}^{m} d_{j}z_i*\frac{L}{1+e^{-w(e_{ij}-q)}}) +
\gamma(\sum_{i=1}^{n} \sum_{k=1}^{q} \frac{1}{a_{ik}+\epsilon} * z_i) +
\kappa(\sum_{i=1}^{n} \sum_{k=1}^{q} v_{k}z_i*\frac{L}{1+e^{-w(a_{ik}-q)}})
\\

\text{s.t.} \quad \sum_{i = 1}^{n} z_i <= 5\\

e_{ij} >= 0.1z_i \quad \forall i,j\\

z_i\in\{0,1\}

$$



\begin{equation*}
\begin{split}
\max_z & \quad \alpha(\sum_{i=1}^{n} \sum_{j=1}^{m} \frac{1}{e_{ij}+\epsilon}\ z_i)+ \beta(\sum_{i=1}^{n} \sum_{j=1}^{m} d_jz_i\frac{L}{1+e^{-w(e_{ij}-q)}}) + \gamma(\sum_{i=1}^{n} \sum_{k=1}^{q} \frac{1}{a_{ik}+\epsilon}z_i) + \kappa(\sum_{i=1}^{n} \sum_{k=1}^{q}v_kz_i \frac{L}{1+e^{-w(a_{ik}-q)}}) \\
\end{split}
\end{equation*}

\begin{equation*}
\begin{split}
\text{s.t.} & \quad \sum_{i=1}^{n}z_i <= 5
\end{split}
\end{equation*}

\begin{equation*}
\begin{split}
\text{s.t.} & \quad e_{ij} >= 0.1z_i \quad \forall i,j
\end{split}
\end{equation*}

\begin{equation*}
\begin{split}
\text{s.t.} & \quad z_i\in\{0,1\}
\end{split}
\end{equation*}

In [2]:
###FOURTH VERSION OF MODEL

model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "OutputFlag", 0) 

M = 1_000_000

# i - candidate stations
# j - current stations
# k - attractions

#logistic curve
L = 1
w = -100
q = 0.3
alpha = 0.5#0.0000004*6
beta = 8#6
gamma = 0.000003#0.0000012
kappa = 2.5#1

@variable(model, z[i=1:N_CANDIDATE_STATIONS], Bin) # z: binary variable to indicate whether or not station is selected

# add constraint that two candidate stations cannot be too close to each other or existing stations

@constraint(model, sum(z[i] for i in 1:N_CANDIDATE_STATIONS) <= N_NEW_STATIONS) #can add up to N new stations
@constraint(model, [i=1:N_CANDIDATE_STATIONS, j=1:N_CURRENT_STATIONS], existing_distance[i, j] >= 0.1 * z[i]) #distance between existing and candidate stations must be at least 400 meters CHECK VALUE

@objective(model, Max, 
    alpha * sum((1/(existing_distance[i, j]+0.0000001)) * z[i] for i=1:N_CANDIDATE_STATIONS, j=1:N_CURRENT_STATIONS) +
    beta * sum(demand[j] * z[i] * (L/(1+exp(-w*(existing_distance[i,j]-q)))) for i=1:N_CANDIDATE_STATIONS, j=1:N_CURRENT_STATIONS) + #enforce that demand for stations further away from candidate stations count less
    gamma * sum((1/(attraction_distance[i, k]+0.0000001)) * z[i] for i=1:N_CANDIDATE_STATIONS, k=1:N_ATTRACTIONS) + 
    kappa * sum(attraction_values[k] * z[i] * (L/(1+exp(-w*(attraction_distance[i,k]-q)))) for i=1:N_CANDIDATE_STATIONS, k=1:N_ATTRACTIONS))

optimize!(model)

optimal_cost = objective_value(model)
stations_chosen = value.(z)

println("Optimal stations: $stations_chosen")


# printing optimal cost
pretty_optimal_cost = round(optimal_cost)
println("Total Cost: $pretty_optimal_cost")

LoadError: UndefVarError: Gurobi not defined

In [6]:
candidate_addresses[findall(x -> x==1, stations_chosen)]

5-element Vector{Any}:
  48.0
  54.0
  80.0
  87.0
 106.0

In [7]:
candidate_addresses_raw[findall(x -> x==1, stations_chosen)]

5-element Vector{Any}:
 "20 Harbor St, Salem, MA 01970, USA"
 "15 Chestnut St, Salem, MA 01970, USA"
 "23 Derby St, Salem, MA 01970, USA"
 "41 Washington Square, Salem, MA 01970, USA"
 "89 Bridge St, Salem, MA 01970, USA"

In [35]:
###ROBUST VERSION OF MODEL

model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "OutputFlag", 0) 

M = 1_000_000

# i - candidate stations
# j - current stations
# k - attractions

#logistic curve
L = 1
w = -100
q = 0.3
alpha = 0.5
beta = 8
gamma = 0.000003
kappa = 2.5

#robust
rho = 0.3

@variable(model, z[i=1:N_CANDIDATE_STATIONS], Bin) # z: binary variable to indicate whether or not station is selected
@variable(model, t)
@variable(model, g[i=1:N_CANDIDATE_STATIONS])

# add constraint that two candidate stations cannot be too close to each other or existing stations

@constraint(model, sum(z[i] for i in 1:N_CANDIDATE_STATIONS) <= N_NEW_STATIONS) #can add up to N new stations
@constraint(model, [i=1:N_CANDIDATE_STATIONS, j=1:N_CURRENT_STATIONS], existing_distance[i, j] >= 0.1 * z[i]) #distance between existing and candidate stations must be at least 400 meters CHECK VALUE
#@constraint(model, [i=1:N_CANDIDATE_STATIONS, j=1:N_CURRENT_STATIONS], sum(-demand[j] * z[i] * (L/(1+exp(-w*(existing_distance[i,j]-q)))) for i=1:N_CANDIDATE_STATIONS, j=1:N_CURRENT_STATIONS)+ (rho * sum(g[i] for i=1:N_CANDIDATE_STATIONS)) <= -t)#+ (rho * norm(z[i], 1)) <= -t) #robust constraint
@constraint(model, [i=1:N_CANDIDATE_STATIONS, j=1:N_CURRENT_STATIONS], sum(-demand[j] * z[i] * (L/(1+exp(-w*(existing_distance[i,j]-q)))) for i=1:N_CANDIDATE_STATIONS, j=1:N_CURRENT_STATIONS)+ (rho * sum(g[i] * (L/(1+exp(-w*(existing_distance[i,j]-q)))))) <= -t)
@constraint(model, [i=1:N_CANDIDATE_STATIONS, j=1:N_CURRENT_STATIONS], g[i] >= z[i]*(L/(1+exp(-w*(existing_distance[i,j]-q)))))
@constraint(model, [i=1:N_CANDIDATE_STATIONS, j=1:N_CURRENT_STATIONS], g[i] >= -z[i]*(L/(1+exp(-w*(existing_distance[i,j]-q)))))

@objective(model, Max, 
    alpha * sum((1/(existing_distance[i, j]+0.0000001)) * z[i] for i=1:N_CANDIDATE_STATIONS, j=1:N_CURRENT_STATIONS) +
   # beta * sum(demand[j] * z[i] * (L/(1+exp(-w*(existing_distance[i,j]-q)))) for i=1:N_CANDIDATE_STATIONS, j=1:N_CURRENT_STATIONS) + #enforce that demand for stations further away from candidate stations count less
    beta * (t) +
    gamma * sum((1/(attraction_distance[i, k]+0.0000001)) * z[i] for i=1:N_CANDIDATE_STATIONS, k=1:N_ATTRACTIONS) + 
    kappa * sum(attraction_values[k] * z[i] * (L/(1+exp(-w*(attraction_distance[i,k]-q)))) for i=1:N_CANDIDATE_STATIONS, k=1:N_ATTRACTIONS))

optimize!(model)

optimal_cost = objective_value(model)
stations_chosen = value.(z)

println("Optimal stations: $stations_chosen")


# printing optimal cost
pretty_optimal_cost = round(optimal_cost)
println("Total Cost: $pretty_optimal_cost")

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19


LoadError: UndefVarError: j not defined

In [34]:
candidate_addresses[findall(x -> x==1, stations_chosen)]

5-element Vector{Any}:
  48.0
  54.0
  80.0
  87.0
 106.0